In [1]:
import jax
import jax.numpy as jnp
import flax.linen as nn
import numpy as np
import chex
import optax
from flax.linen.initializers import constant, orthogonal
from typing import Sequence, NamedTuple, Any, Dict
from flax.training.train_state import TrainState
import distrax

from jaxmarl import make
from jaxmarl.wrappers.baselines import (
    SMAXLogWrapper,
    MPELogWrapper,
    LogWrapper,
    CTRolloutManager,
)
from jaxmarl.environments.smax import map_name_to_scenario, HeuristicEnemySMAX
from jaxmarl.viz.visualizer import Visualizer, SMAXVisualizer

import functools
from functools import partial
import time
import os
import matplotlib.pyplot as plt

from qdax.core.map_elites import MAPElites
from qdax.core.containers.mapelites_repertoire import compute_cvt_centroids, MapElitesRepertoire
#import qdax.tasks.brax.v1 as environments
#from qdax.tasks.brax.v1.env_creators import scoring_function_brax_envs as scoring_function
from qdax.core.neuroevolution.buffers.buffer import QDTransition
from qdax.core.neuroevolution.networks.networks import MLP
from qdax.core.emitters.mutation_operators import isoline_variation
from qdax.core.emitters.standard_emitters import MixingEmitter
from qdax.utils.plotting import plot_map_elites_results
from qdax.utils.metrics import CSVLogger, default_qd_metrics

## Brax version conflict with JaxMARL(0.10.3) and QDax(0.10.4 / 0.12.3)

In [2]:
from typing import Callable, Optional, Tuple
from qdax.custom_types import (
    Descriptor,
    EnvState,
    ExtraScores,
    Fitness,
    Genotype,
    Observation,
    Params,
    RNGKey,
)

In [ ]:
config = {
    # valid for iql, vdn, qmix
    "TOTAL_TIMESTEPS": 1e7,
    "NUM_ENVS": 1, #16,
    "NUM_STEPS": 128,
    "BUFFER_SIZE": 10000, #5000,
    "BUFFER_BATCH_SIZE": 32,
    "HIDDEN_SIZE": 128, #512,
    "MIXER_EMBEDDING_DIM": 64,
    "MIXER_HYPERNET_HIDDEN_DIM": 256,
    "MIXER_INIT_SCALE": 0.001,
    "EPS_START": 1.0,
    "EPS_FINISH": 0.05,
    "EPS_DECAY": 0.1, # percentage of updates
    "MAX_GRAD_NORM": 10,
    "TARGET_UPDATE_INTERVAL": 5, #10,
    "TAU": 1.,
    "NUM_EPOCHS": 8,
    "LR": 0.00005,
    "LEARNING_STARTS": 10000, # timesteps
    "LR_LINEAR_DECAY": False,
    "GAMMA": 0.99,
    "REW_SCALE": 10., # scale the reward to the original scale of SMAC

    # ENV
    "ENV_NAME": "HeuristicEnemySMAX",
    #"MAP_NAME": "3s_vs_5z",
    "MAP_NAME": "2s3z",
    "ENV_KWARGS": {
        "see_enemy_actions": True,
        "walls_cause_death": True,
        "attack_mode": "closest",
    },

    "NUM_SEEDS": 1, # number of vmapped seeds
    "SEED": 88,

    "HYP_TUNE": False, # perform hyp tune

    # evaluate
    "TEST_DURING_TRAINING": False, #True,
    "TEST_INTERVAL": 0.05, # as a fraction of updates, i.e. log every 5% of training process
    "TEST_NUM_STEPS": 128,
    "TEST_NUM_ENVS": 512, # number of episodes to average over, can affect performance
}
batch_size = 64# 128 # Num of offsprings 
#env_name = 'walker2d_uni'
episode_length = config["NUM_STEPS"] #128  # NUM_STEPS
num_iterations = int(config["TOTAL_TIMESTEPS"] / (config["NUM_ENVS"] * batch_size * config["NUM_STEPS"]))
seed = 88 
policy_hidden_layer_sizes = (128, 128, 128) #(64, 64)
iso_sigma = 0.05 #0.005 
line_sigma = 0.5 #0.05 
num_init_cvt_samples = 20000 #50000 
num_centroids = 2000 #1024 
min_descriptor = 0. 
max_descriptor = 1.0 
number_of_descriptors=2



In [4]:
class ScannedRNN(nn.Module):

    @partial(
        nn.scan,
        variable_broadcast="params",
        in_axes=0,
        out_axes=0,
        split_rngs={"params": False},
    )
    @nn.compact
    def __call__(self, carry, x):
        """Applies the module."""
        rnn_state = carry
        ins, resets = x
        hidden_size = ins.shape[-1]
        rnn_state = jnp.where(
            resets[:, np.newaxis],
            self.initialize_carry(hidden_size, *ins.shape[:-1]),
            rnn_state,
        )
        new_rnn_state, y = nn.GRUCell(hidden_size)(rnn_state, ins)
        return new_rnn_state, y

    @staticmethod
    def initialize_carry(hidden_size, *batch_size):
        # Use a dummy key since the default state init fn is just zeros.
        return nn.GRUCell(hidden_size, parent=None).initialize_carry(
            jax.random.PRNGKey(0), (*batch_size, hidden_size)
        )


class RNNQNetwork(nn.Module):
    # homogenous agent for parameters sharing, assumes all agents have same obs and action dim
    action_dim: int
    hidden_dim: int
    init_scale: float = 1.0

    @nn.compact
    def __call__(self, hidden, obs, dones):
        embedding = nn.Dense(
            self.hidden_dim,
            kernel_init=orthogonal(self.init_scale),
            bias_init=constant(0.0),
        )(obs)
        embedding = nn.relu(embedding)

        rnn_in = (embedding, dones)
        hidden, embedding = ScannedRNN()(hidden, rnn_in)

        q_vals = nn.Dense(
            self.action_dim,
            kernel_init=orthogonal(self.init_scale),
            bias_init=constant(0.0),
        )(embedding)

        return hidden, q_vals

In [5]:
# Init environment
#env = environments.create(env_name, episode_length=episode_length)
scenario = map_name_to_scenario(config["MAP_NAME"])
env = HeuristicEnemySMAX(scenario=scenario, **config["ENV_KWARGS"])
env = SMAXLogWrapper(env)
wrapped_env = CTRolloutManager(env, batch_size=config["NUM_ENVS"])

rng = jax.random.PRNGKey(config["SEED"])
reset_fn = jax.jit(wrapped_env.batch_reset)
#reset_fn = jax.jit(env.reset)

config["NUM_ACTORS"] = env.num_agents * config["NUM_ENVS"]
config["NUM_UPDATES"] = (
    config["TOTAL_TIMESTEPS"] // config["NUM_STEPS"] // config["NUM_ENVS"]
)


# Init a random key
key = jax.random.key(seed)

# Init policy network
policy_layer_sizes = policy_hidden_layer_sizes + (env.action_space(env.agents[0]).n,) # Not actually used
#policy_network = ActorRNN(env.action_space(env.agents[0]).n, config=config)
policy_network = RNNQNetwork(
            action_dim=wrapped_env.max_action_space,
            hidden_dim=config["HIDDEN_SIZE"],
        )

# Init population of controllers
key, subkey = jax.random.split(key)
#keys = jax.random.split(subkey, num=batch_size)
# fake_batch = jnp.zeros(shape=(batch_size, env.observation_size)) # Not needed as on-policy is used for no batches?
# init_variables = jax.vmap(policy_network.init)(keys, fake_batch)
init_x = (
    jnp.zeros(
        (1, 1, wrapped_env.obs_size)
    ),  # (time_step, batch_size, obs_size)
    jnp.zeros((1, 1)),  # (time_step, batch size)
)
init_hstate = ScannedRNN.initialize_carry(
    config["HIDDEN_SIZE"], 1
)  # (batch_size, hidden_dim)

keys = jax.random.split(subkey, batch_size)
# Assume init_hstate: [H], init_x: [X]
fake_hstate = jnp.stack([init_hstate] * batch_size)  # [N, H]
#fake_x = jnp.stack([init_x] * config["NUM_ENVS"])            # [N, X]
fake_x = tuple(jnp.stack([x] * batch_size) for x in init_x)

#network_params = policy_network.init(subkey, init_hstate, init_x)
network_params = jax.vmap(
    policy_network.init
)(keys, fake_hstate, *fake_x)

In [6]:
#class Transition(NamedTuple):
#    global_done: jnp.ndarray
#    done: jnp.ndarray
#    action: jnp.ndarray
#    reward: jnp.ndarray
#    obs: jnp.ndarray
#    env_state: jnp.ndarray
#    info: jnp.ndarray
#    avail_actions: jnp.ndarray

class Transition(NamedTuple):
    global_done: jnp.ndarray
    reward: jnp.ndarray
    env_state: jnp.ndarray

@chex.dataclass(frozen=True)
class Timestep:
    obs: dict
    actions: dict
    rewards: dict
    dones: dict
    avail_actions: dict


class CustomTrainState(TrainState):
    target_network_params: Any
    timesteps: int = 0
    n_updates: int = 0
    grad_steps: int = 0

def batchify(x: dict):
    return jnp.stack([x[agent] for agent in env.agents], axis=0)

def unbatchify(x: jnp.ndarray):
    return {agent: x[i] for i, agent in enumerate(env.agents)}

In [7]:
# Define the function to play a step with the policy in the environment
def play_step_fn(
    runner_state
):
    """
    Play an environment step and return the updated state and the transition.
    """
    #hs, last_obs, last_dones, env_state, rng = carry
    policy_params, env_state, last_obs, last_dones, hstate, rng = runner_state

    rng, rng_a, rng_s = jax.random.split(rng, 3)

    # (num_agents, 1 (dummy time), num_envs, obs_size)
    _obs = batchify(last_obs)[:, np.newaxis]
    _dones = batchify(last_dones)[:, np.newaxis]
    #_obs = batchify_multi(last_obs)[:, :, np.newaxis]    # [batch, num_agents, 1, num_envs, obs_dim]
    #_dones = batchify_multi(last_dones)[:, :, np.newaxis] # [batch, num_agents, 1, num_envs]

    print("hstate shape:", hstate.shape)  # should be [BATCH_SIZE, NUM_ENVS, NUM_AGENTS, HIDDEN_SIZE]
    print("_obs shape:", _obs.shape)      # should be [BATCH_SIZE, NUM_ENVS, OBS_DIM]
    print("_dones shape:", _dones.shape)  # should be [BATCH_SIZE, NUM_ENVS, OBS_DIM]

    new_hstate, q_vals = jax.vmap(
        policy_network.apply, in_axes=(None, 0, 0, 0)
    )(  # vmap across the agent dim
        policy_params,
        hstate,
        _obs,
        _dones,
    )
    print("new_hstate shape:", new_hstate.shape)  # should be [BATCH_SIZE, NUM_ENVS, NUM_AGENTS, HIDDEN_SIZE]

    q_vals = q_vals.squeeze(
        axis=1
    )  # (num_agents, num_envs, num_actions) remove the time dim

    # explore
    avail_actions = wrapped_env.get_valid_actions(env_state.env_state)
    avail_actions_batchified = batchify(avail_actions)

    #eps = eps_scheduler(train_state.n_updates)
    #_rngs = jax.random.split(rng_a, env.num_agents)
    #actions = jax.vmap(eps_greedy_exploration, in_axes=(0, 0, None, 0))(
    #    _rngs, q_vals, eps, batchify(avail_actions)
    #)
    
    unavail_actions = 1 - avail_actions_batchified
    q_vals = q_vals - (unavail_actions * 1e10)
    actions_array = jnp.argmax(q_vals, axis=-1)
    actions_dict = unbatchify(actions_array)
    print("actions", actions_dict)


    new_obs, new_env_state, rewards, dones, infos = wrapped_env.batch_step(
        rng_s, env_state, actions_dict
    )
    
    timestep = Timestep(
        obs=last_obs,
        actions=actions_dict,
        rewards=jax.tree.map(lambda x:config.get("REW_SCALE", 1)*x, rewards),
        dones=last_dones,
        avail_actions=avail_actions,
    )

    #per_agent_dones = jnp.stack(
    #    [dones[agent] for agent in env.agents], axis=-1
    #)  # shape: [num_envs, num_agents]

    transition = Transition(
        jnp.tile(dones["__all__"], env.num_agents),
        #last_dones,
        #actions_array.squeeze(),
        batchify(rewards).reshape(-1),#.reshape(batch_size, config["NUM_ENVS"], -1),  #batchify(rewards, env.agents, config["NUM_ACTORS"]).squeeze(),
        #_obs,
        env_state.env_state.state,
        #infos,
        #avail_actions,
    )

    runner_state = (policy_params, new_env_state, new_obs, dones, new_hstate, rng)
    
    return runner_state, (transition, timestep)

In [8]:
def generate_unroll(
    runner_state,
    episode_length: int,
    play_step_fn: Callable[
        [EnvState, Params, RNGKey],
        Tuple[
            EnvState,
            Params,
            RNGKey,
            Transition,
        ],
    ],
) -> Tuple[EnvState, Transition]:
    """Generates an episode according to the agent's policy, returns the final state of
    the episode and the transitions of the episode.

    Args:
        init_state: first state of the rollout.
        policy_params: params of the individual.
        key: random key for stochasiticity handling.
        episode_length: length of the rollout.
        play_step_fn: function describing how a step need to be taken.

    Returns:
        A new state, the experienced transition.
    """

    def _scan_play_step_fn(
        carry, unused_arg: Any #: Tuple[EnvState, Params, RNGKey]
    ) -> Tuple[Tuple[EnvState, Params, RNGKey], Transition]:
        runner_state, (transitions, timestep) = play_step_fn(carry)
        return runner_state, (transitions, timestep)

    runner_state, (transitions, timestep) = jax.lax.scan(
        _scan_play_step_fn,
        runner_state,
        (),
        length=episode_length,
    )
    return runner_state, (transitions, timestep)


def get_mask_from_transitions(
    data: Transition,
) -> jnp.ndarray:
    is_done = jnp.clip(jnp.cumsum(data.global_done, axis=1), 0, 1)
    mask = jnp.roll(is_done, 1, axis=1)
    mask = mask.at[:, 0].set(0)
    return mask

In [9]:
def scoring_function(
    policy_params: Genotype,
    key: RNGKey,
    episode_length: int,
    play_reset_fn: Callable[[RNGKey], EnvState],
    play_step_fn: Callable[
        [EnvState, Params, RNGKey], Tuple[EnvState, Params, RNGKey, QDTransition]
    ],
    descriptor_extractor: Callable[[QDTransition, jnp.ndarray], Descriptor],
) -> Tuple[Fitness, Descriptor, ExtraScores]:
    """Evaluates policies contained in policies_params in parallel.
    The play_reset_fn function allows for a more general scoring_function that can be
    called with different batch-size and not only with a batch-size of the same
    dimension as init_states.

    To define purely stochastic environments, using the reset function from the
    environment, use "play_reset_fn = env.reset".

    To define purely deterministic environments, as in "scoring_function", generate
    a single init_state using "init_state = env.reset(key)", then use
    "play_reset_fn = lambda key: init_state".

    Args:
        policies_params: The parameters of closed-loop controllers/policies to evaluate.
        key: A jax random key
        episode_length: The maximal rollout length.
        play_reset_fn: The function to reset the environment and obtain initial states.
        play_step_fn: The function to play a step of the environment.
        descriptor_extractor: The function to extract the descriptor.

    Returns:
        fitness: Array of fitnesses of all evaluated policies
        descriptor: Behavioural descriptors of all evaluated policies
        extra_scores: Additional information resulting from the evaluation
    """

    # Reset environments
    key, subkey = jax.random.split(key)
    #keys = jax.random.split(subkey, jax.tree.leaves(policies_params)[0].shape[0])
    #keys = jax.random.split(subkey, config["NUM_ENVS"])
    #init_obs, init_env_state = jax.vmap(play_reset_fn, in_axes=(0,))(keys)
    init_obs, init_env_state = play_reset_fn(key)
    #print("init_obs is a dict with keys:", init_obs.keys())
    #for k, v in init_obs.items():
    #    print(f"init_obs[{k}] shape:", v.shape) 
    
    keys = jax.random.split(key, batch_size)

    #init_dones = {
    #    agent: jnp.zeros((config["NUM_ENVS"]), dtype=bool)
    #    for agent in env.agents + ["__all__"]
    #}
    init_hstate = ScannedRNN.initialize_carry(
        config["HIDDEN_SIZE"], len(env.agents), config["NUM_ENVS"]
    )
    
    batched_env_state = jax.tree.map(lambda x: jnp.stack([x] * batch_size), init_env_state)
    batched_obs = jax.tree.map(lambda x: jnp.stack([x] * batch_size), init_obs)
    batched_dones = {
        agent: jnp.stack([jnp.zeros(config["NUM_ENVS"], dtype=bool)] * batch_size)
        for agent in env.agents + ["__all__"]
    }
    batched_hstate = jnp.stack([init_hstate] * batch_size)

    #print("batched_obs is a dict with keys:", batched_obs.keys())
    #for k, v in batched_obs.items():
    #    print(f"batched_obs[{k}] shape:", v.shape) 

    init_runner_state = (policy_params, batched_env_state, batched_obs, batched_dones, batched_hstate, keys)
    #print("init_runner_state:")
    #jax.tree.map(lambda x: print(x.shape), init_runner_state)
    
    # Step environments
    unroll_fn = functools.partial(
        generate_unroll,
        episode_length=episode_length,
        play_step_fn=play_step_fn,
    )
    #keys = jax.random.split(key, jax.tree.leaves(policies_params)[0].shape[0])
    _, (data, timestep) = jax.vmap(unroll_fn)(init_runner_state) # data = Transistions data struc
    #jax.debug.print("after vmap timestep shape: {}", 
    #jax.tree.map(lambda x: x.shape, timestep))

    # Create a mask to extract data properly
    mask = get_mask_from_transitions(data)

    # Evaluate
    print("data.reward shape:", data.reward.shape)
    print("data.global_done shape:", data.global_done.shape)
    print("mask shape:", mask.shape)
    #fitnesses = jnp.sum(data.reward * (1.0 - mask), axis=1)
    fitnesses = jnp.sum(data.reward * (1.0 - mask), axis=(1, 2))  # shape (32,)
    fitnesses = fitnesses.reshape(-1, 1)  # shape (32, 1)
    descriptors = descriptor_extractor(data, mask)

    print("fitness shape:", fitnesses.shape)
    print("descriptors shape:", descriptors.shape)

    return fitnesses, descriptors, {"transitions": timestep}#data}


In [10]:
def compute_descriptors(data):
    """
    Compute per-trajectory descriptors:
    - % of units that moved last step (per batch item)
    - % of units that attacked last step (per batch item)
    - Avg ally-ally distance at final step (per batch item)
    
    Output: shape (batch_size, 3)
    """
    movement_actions = data.env_state.prev_movement_actions          # (B, T, E, A, 2)
    attack_actions = data.env_state.prev_attack_actions              # (B, T, E, A)
    alive = data.env_state.unit_alive.astype(jnp.float32)            # (B, T, E, A)
    positions = data.env_state.unit_positions                        # (B, T, E, A, 2)
    teams = data.env_state.unit_teams                                # (B, T, E, A)

    moved = jnp.any(movement_actions != 0.0, axis=-1)                # (B, T, E, A)
    attacked = attack_actions > 0                                    # (B, T, E, A)
    is_ally = (teams == 0).astype(jnp.float32)                        # (B, T, E, A)
    ally_alive = alive * is_ally                                      # (B, T, E, A)

    # Per batch movement/attack % (sum over all but batch dim)
    def pct_nonzero(mask, alive_mask):
        numerator = jnp.sum(mask * alive_mask, axis=(1, 2, 3))        # (B,)
        denominator = jnp.sum(alive_mask, axis=(1, 2, 3)) + 1e-8      # (B,)
        return numerator / denominator                                # (B,)

    movement_pct = pct_nonzero(moved, ally_alive)                    # (B,)
    attack_pct = pct_nonzero(attacked, ally_alive)                   # (B,)

    # Final step only for positions and masks
    final_positions = positions[:, -1, :, :, :]                      # (B, E, A, 2)
    final_alive = alive[:, -1, :, :]                                 # (B, E, A)
    final_teams = teams[:, -1, :, :]                                 # (B, E, A)
    ally_mask = (final_teams == 0) * final_alive                     # (B, E, A)

    def avg_pairwise_distance(pos, mask):
        """
        pos: (A, 2)
        mask: (A,)
        returns: scalar distance
        """
        diffs = pos[:, None, :] - pos[None, :, :]                    # (A, A, 2)
        dists = jnp.linalg.norm(diffs, axis=-1)                      # (A, A)
        pair_mask = mask[:, None] * mask[None, :]                    # (A, A)
        not_self = 1 - jnp.eye(mask.shape[0])
        valid_pairs = pair_mask * not_self
        total_dist = jnp.sum(dists * valid_pairs)
        num_pairs = jnp.sum(valid_pairs)
        return jnp.where(num_pairs > 0, total_dist / num_pairs, 0.0)

    # vmap over batch and envs: returns shape (B, E)
    avg_dists = jax.vmap(
        jax.vmap(avg_pairwise_distance, in_axes=(0, 0)),
        in_axes=(0, 0)
    )(final_positions, ally_mask)

    avg_dist = jnp.mean(avg_dists, axis=1)  # mean over envs → shape (B,)

    # Normalize and clip
    max_distance = 32
    quarter_max = 0.25 * max_distance 
    norm_dist = avg_dist / quarter_max
    norm_dist = jnp.clip(norm_dist, 0.0, 1.0)

    # Stack all descriptors → shape (B, 3)
    #return jnp.stack([movement_pct, attack_pct, norm_dist], axis=-1)
    return jnp.stack([movement_pct, norm_dist], axis=-1)



def descriptor_extraction_fn(data: Transition, mask: jnp.ndarray) -> Descriptor:
    """Compute final xy position.

    This function suppose that state descriptor is the xy position, as it
    just select the final one of the state descriptors given.
    """
    # reshape mask for descriptor extraction
    mask = jnp.expand_dims(mask, axis=-1)

    # Get descriptor
    #last_index = jnp.int32(jnp.sum(1.0 - mask, axis=1)) - 1
    #descriptors = jax.vmap(lambda x, y: x[y])(data.state_desc, last_index)
    descriptors = compute_descriptors(data)

    # remove the dim coming from the trajectory
    return descriptors#.squeeze(axis=1)


In [11]:
from functools import partial
from dataclasses import dataclass

from qdax.core.emitters.repertoire_selectors.selector import Selector
from qdax.core.emitters.qpg_emitter import QualityPGConfig, QualityPGEmitterState
from qdax.tasks.brax.v1.envs.base_env import QDEnv


from qdax.core.emitters.multi_emitter import MultiEmitter
from qdax.core.containers.ga_repertoire import GARepertoire
from qdax.core.containers.repertoire import Repertoire
#from qdax.core.neuroevolution.losses.td3_loss import make_td3_loss_fn
from qdax.core.emitters.emitter import Emitter
from qdax.core.neuroevolution.networks.networks import QModule
from qdax.core.neuroevolution.buffers.buffer import QDTransition, ReplayBuffer

from qdax.core.emitters.pga_me_emitter import PGAMEConfig, PGAMEEmitter

import flashbax as fbx

In [12]:
def make_iql_loss_fn(
        policy_fn: Callable[[Params, Observation], jnp.ndarray],
        reward_scaling: float,
        discount: float,
) -> Callable[[Params, Transition], jnp.ndarray]:
    """Creates the loss functions for IQL.

    Args:
        policy_fn: forward pass through the neural network defining the policy.
        reward_scaling: value to multiply the reward given by the environment.
        discount: discount factor.

    Returns:
        Return the loss functions used to train the policy.
    """
    def _policy_loss_fn(
        policy_params: Params,
        target_policy_params: Params,
        minibatch: Transition,
        #emitter_state: CustomQualityPGEmitterState,
    ) -> jnp.ndarray:
        #minibatch = emitter_state.buffer.sample(emitter_state.buffer_state, emitter_state.key).experience

        minibatch = jax.tree.map(
            lambda x: jnp.swapaxes(
                x[:, 0], 0, 1
            ),  # remove the dummy sequence dim (1) and swap batch and temporal dims
            minibatch,
        )  # (max_time_steps, batch_size, ...)

        # preprocess network input
        init_hs = ScannedRNN.initialize_carry(
            config["HIDDEN_SIZE"],
            len(env.agents),
            config["BUFFER_BATCH_SIZE"],
        )
        # num_agents, timesteps, batch_size, ...
        _obs = batchify(minibatch.obs)
        _dones = batchify(minibatch.dones)
        _actions = batchify(minibatch.actions)
        _rewards = batchify(minibatch.rewards)
        _avail_actions = batchify(minibatch.avail_actions)
        ###################################################################################################
        #print("_obs.shape", _obs.shape)
        #print("_does.shape", _dones.shape)
        #print("init_hs.shape", init_hs.shape)
        ###################################################################################################
        _, q_next_target = jax.vmap(policy_fn, in_axes=(None, 0, 0, 0))(
            target_policy_params, #train_state.target_network_params,
            init_hs,
            _obs,
            _dones,
        )  # (num_agents, timesteps, batch_size, num_actions)

        _, q_vals = jax.vmap(policy_fn, in_axes=(None, 0, 0, 0))(
            policy_params,
            init_hs,
            _obs,
            _dones,
        )  # (num_agents, timesteps, batch_size, num_actions)

        # get logits of the chosen actions
        chosen_action_q_vals = jnp.take_along_axis(
            q_vals,
            _actions[..., np.newaxis],
            axis=-1,
        ).squeeze(
            -1
        )  # (num_agents, timesteps, batch_size,)

        unavailable_actions = 1 - _avail_actions
        valid_q_vals = q_vals - (unavailable_actions * 1e10)

        # get the q values of the next state
        q_next = jnp.take_along_axis(
            q_next_target,
            jnp.argmax(valid_q_vals, axis=-1)[..., np.newaxis],
            axis=-1,
        ).squeeze(
            -1
        )  # (num_agents, timesteps, batch_size,)

        target = (
            _rewards[:, :-1] * reward_scaling
            + (1 - _dones[:, :-1]) * discount * q_next[:, 1:]
        )

        chosen_action_q_vals = chosen_action_q_vals[:, :-1]
        loss = jnp.mean(
            (chosen_action_q_vals - jax.lax.stop_gradient(target)) ** 2
        )

        #return loss, chosen_action_q_vals.mean()
        return loss
    
    return _policy_loss_fn

#class CustomQualityPGEmitterState(QualityPGEmitterState):
    #target_network_params: Any
#    buffer_state: Any

@chex.dataclass(frozen=True)
class CustomQualityPGEmitterState:
    #target_network_params: Any
    buffer_state: Any
    #replay_buffer: Any = struct.field(pytree_node=False)  # Static Python object
    key: RNGKey

In [ ]:
class myQualityPGEmitter(Emitter):
    """
    A policy gradient emitter used to implement the Policy Gradient Assisted MAP-Elites
    (PGA-Map-Elites) algorithm.
    """

    def __init__(
        self,
        config: QualityPGConfig,
        policy_network: nn.Module,
        env: QDEnv,
        selector: Optional[Selector] = None,
    ) -> None:
        self._config = config
        self._env = env
        self._selector = selector
        self._actor_critic_iterations = int(
            config.num_critic_training_steps / config.policy_delay
        )  # actor and critic training are packed into a single function

        # Init Critics
        #critic_network = QModule(
        #    n_critics=2, hidden_layer_sizes=self._config.critic_hidden_layer_size
        #)
        #critic_network = CriticRNN(action_dim=env.action_space(env.agents[0]).n, config=config, n_critics=2)
        #target_q_network = RNNQNetwork(
        #    action_dim=wrapped_env.max_action_space,
        #    hidden_dim=config["HIDDEN_SIZE"],
        #)
        
        #self._critic_network = target_q_network #critic_network

        # Set up the losses and optimizers - return the opt states
        self._policy_loss_fn = make_iql_loss_fn( #make_td3_loss_fn(
            policy_fn=policy_network.apply,
            #critic_fn=critic_network.apply,
            reward_scaling=self._config.reward_scaling,
            discount=self._config.discount,
            #noise_clip=self._config.noise_clip,
            #policy_noise=self._config.policy_noise,
        )

        # Init optimizers
        #self._actor_optimizer = optax.adam(
        #    learning_rate=self._config.actor_learning_rate
        #)
        #self._critic_optimizer = optax.adam(
        #    learning_rate=self._config.critic_learning_rate
        #)
        #self._policies_optimizer = optax.adam(
        #    learning_rate=self._config.policy_learning_rate
        #)
        
        #lr_scheduler = optax.linear_schedule(
        #    init_value=config["LR"],
        #    end_value=1e-10,
        #    transition_steps=(config["NUM_EPOCHS"]) * config["NUM_UPDATES"],
        #)
        #lr = lr_scheduler if config.get("LR_LINEAR_DECAY", False) else config["LR"]
        #tx = optax.chain(
        #    optax.clip_by_global_norm(config["MAX_GRAD_NORM"]),
        #    optax.radam(learning_rate=lr),
        #)
        #self._policies_optimizer = tx

        buffer = fbx.make_trajectory_buffer(
            max_length_time_axis=self._config.replay_buffer_size // batch_size,
            min_length_time_axis=self._config.batch_size,
            sample_batch_size=self._config.batch_size,
            add_batch_size=self._config.num_envs * batch_size,
            sample_sequence_length=1,
            period=1,
        )
        self._buffer = buffer

        # Init optimizers
        self._policies_optimizer = optax.adam(
            learning_rate=self._config.policy_learning_rate
        )

    @property
    def batch_size(self) -> int:
        """
        Returns:
            the batch size emitted by the emitter.
        """
        return self._config.env_batch_size

    @property
    def use_all_data(self) -> bool:
        """Whether to use all data or not when used along other emitters.

        QualityPGEmitter uses the transitions from the genotypes that were generated
        by other emitters.
        """
        return True

    def init(
        self,
        key: RNGKey,
        repertoire: Repertoire,
        genotypes: Genotype,
        fitnesses: Fitness,
        descriptors: Descriptor,
        extra_scores: ExtraScores,
    ) -> CustomQualityPGEmitterState:
        """Initializes the emitter state.

        Args:
            genotypes: The initial population.
            key: A random key.

        Returns:
            The initial state of the PGAMEEmitter.
        """
        '''
        #observation_size = self._env.observation_size
        #action_size = self._env.action_size
        #descriptor_size = self._env.state_descriptor_length

        obs_space = self._env.observation_space(env.agents[0])#.shape[0]
        action_space = self._env.action_space(env.agents[0])#.shape[0]

        observation_size = int(np.prod(obs_space.shape))
        action_size = int(np.prod(action_space.shape))
        descriptor_size = 2 # hardcoded

        # Initialise critic, greedy actor and population
        #key, subkey = jax.random.split(key)
        #fake_obs = jnp.zeros(shape=(observation_size,))
        #fake_action = jnp.zeros(shape=(action_size,))
        #critic_params = self._critic_network.init(
        #    subkey, obs=fake_obs, actions=fake_action
        #)

        #init_hstate = ScannedRNN.initialize_carry(1, config["GRU_HIDDEN_DIM"])
        key, subkey = jax.random.split(key)
        #fake_hstate = jnp.stack([init_hstate] * 1) 
        #fake_obs = tuple(jnp.stack([x] * batch_size) for x in init_x)
        #fake_action = jnp.zeros(shape=(action_size,))
        #critic_params = self._critic_network.init(subkey, fake_hstate, fake_obs, fake_action)

        fake_x = (
            jnp.zeros(
                (1, config["NUM_ENVS"], env.observation_space(env.agents[0]).shape[0])
            ),
            jnp.zeros((1, config["NUM_ENVS"])),
            jnp.zeros((1, config["NUM_ENVS"], env.action_space(env.agents[0]).n)),
        )
        fake_hstate = ScannedRNN.initialize_carry(config["NUM_ENVS"], config["GRU_HIDDEN_DIM"])
        fake_action = jnp.zeros(shape=(action_size,))
        critic_params = self._critic_network.init(subkey, fake_hstate, fake_x, fake_action)

        #target_critic_params = jax.tree.map(lambda x: x, critic_params)

        #actor_params = jax.tree.map(lambda x: x[0], genotypes)
        #target_actor_params = jax.tree.map(lambda x: x[0], genotypes)

        # Prepare init optimizer states
        #critic_optimizer_state = self._critic_optimizer.init(critic_params)
        #actor_optimizer_state = self._actor_optimizer.init(actor_params)

        # Initialize replay buffer
        dummy_transition = QDTransition.init_dummy(
            observation_dim=observation_size,
            action_dim=action_size,
            descriptor_dim=descriptor_size,
        )

        replay_buffer = ReplayBuffer.init(
            buffer_size=self._config.replay_buffer_size, transition=dummy_transition
        )

        # get the transitions out of the dictionary
        assert "transitions" in extra_scores.keys(), "Missing transitions or wrong key"
        transitions = extra_scores["transitions"]

        #print("Transitions type:", type(transitions))
        #print("Transitions fields:", transitions)

        # add transitions in the replay buffer
        replay_buffer = replay_buffer.insert(transitions)

        '''

        ###########################################################################################################################################
        '''
        init_x = (
            jnp.zeros(
                (1, 1, wrapped_env.obs_size)
            ),  # (time_step, batch_size, obs_size)
            jnp.zeros((1, 1)),  # (time_step, batch size)
        )
        init_hs = ScannedRNN.initialize_carry(
            config["HIDDEN_SIZE"], 1
        )  # (batch_size, hidden_dim)
        network_params = policy_network.init(rng, init_hs, *init_x)

        lr_scheduler = optax.linear_schedule(
            init_value=config["LR"],
            end_value=1e-10,
            transition_steps=(config["NUM_EPOCHS"]) * config["NUM_UPDATES"],
        )

        lr = lr_scheduler if config.get("LR_LINEAR_DECAY", False) else config["LR"]

        tx = optax.chain(
            optax.clip_by_global_norm(config["MAX_GRAD_NORM"]),
            optax.radam(learning_rate=lr),
        )

        train_state = CustomTrainState.create(
            apply_fn=policy_network.apply,
            params=network_params,
            target_network_params=network_params,
            tx=tx,
        )
        '''
        # INIT BUFFER
        # to initalize the buffer is necessary to sample a trajectory to know its strucutre
        def _env_sample_step(env_state, unused):
            rng, key_a, key_s = jax.random.split(
                jax.random.PRNGKey(0), 3
            )  # use a dummy rng here
            key_a = jax.random.split(key_a, env.num_agents)
            actions = {
                agent: self._env.batch_sample(key_a[i], agent)
                for i, agent in enumerate(env.agents)
            }
            avail_actions = self._env.get_valid_actions(env_state.env_state)
            obs, env_state, rewards, dones, infos = self._env.batch_step(
                key_s, env_state, actions
            )
            timestep = Timestep(
                obs=obs,
                actions=actions,
                rewards=rewards,
                dones=dones,
                avail_actions=avail_actions,
            )
            return env_state, timestep

        _, _env_state = self._env.batch_reset(rng)
        _, sample_traj = jax.lax.scan(
            _env_sample_step, _env_state, None, self._config.num_steps
        )
        sample_traj_unbatched = jax.tree.map(
            lambda x: x[:, 0], sample_traj
        )  # remove the NUM_ENV dim
        
        # Remove time dimension
        sample_traj_unbatched = jax.tree.map(lambda x: x[:, 0], sample_traj)  # (16, …)
        # Tile across parallel_policies(batch_size) to match real add_batch_size
        #sample_traj_unbatched = jax.tree.map(
        #    lambda x: jnp.repeat(x, batch_size, axis=0),
        #    sample_traj_unbatched
        #)  # now (128, …)
        
        print("Init sample_traj shape:", jax.tree.map(lambda x: x.shape, sample_traj_unbatched))
        buffer_state = self._buffer.init(sample_traj_unbatched)
        
        # get the transitions out of the dictionary
        assert "transitions" in extra_scores.keys(), "Missing transitions or wrong key"
        transitions = extra_scores["transitions"]

        # add transitions in the replay buffer
        #replay_buffer = replay_buffer.insert(transitions)
        # BUFFER UPDATE
        #print("Transitions keys:", transitions.keys())
        #for k, v in transitions.items():
        #    print(k, v.shape, v.dtype)
        
        #jax.tree.map(lambda x: print(x.shape), transitions)
        #jax.tree.map(lambda x: print("Buffer expects:", x.shape), buffer_state.experience)
        
        def prepare_for_buffer(x):
            # x: (P, T, E, *feat_dims)
            P, T, E, *feat_dims = x.shape

            # First move to (P, E, T, *feat_dims)
            x = x.transpose(0, 2, 1, *range(3, x.ndim))

            # Merge P * E into batch
            x = x.reshape(P * E, T, *feat_dims)

            # Add dummy seq_len=1 in the middle -> (batch, 1, T, feat_dim...)
            x = x[:, np.newaxis, ...]

            return x
        
        buffer_traj_batch = jax.tree.map(
            prepare_for_buffer,
            transitions
        )
        #buffer_traj_batch = jax.tree.map(
        #    lambda x: jnp.swapaxes(x, 0, 1)[
        #        :, np.newaxis
        #    ],  # put the batch dim first and add a dummy sequence dim
        #    transitions,
        #)  # (num_envs, 1, time_steps, ...)
        #buffer_traj_batch = jax.tree.map(
        #    lambda x: jnp.swapaxes(
        #        # Reorder from (num_policies, time_steps, num_envs, …)
        #        # → (time_steps, num_policies, num_envs, …)
        #        x.transpose(1, 0, 2, *range(3, x.ndim))
        #        # Merge num_policies * num_envs into one dimension
        #        .reshape(x.shape[1], -1, *x.shape[3:]),
        #        0, 1  # swap (time_steps, merged_envs, …) → (merged_envs, time_steps, …)
        #    )[:, np.newaxis],  # add dummy sequence dim
        #    transitions,
        #)
        print("Runtime sample_traj shape:", jax.tree.map(lambda x: x.shape, buffer_traj_batch))

        buffer_state = self._buffer.add(buffer_state, buffer_traj_batch)


        # Initial training state
        #emitter_state = QualityPGEmitterState(
        #    critic_params=critic_params,
        #    critic_optimizer_state=critic_optimizer_state,
        #    actor_params=actor_params,
        #    actor_opt_state=actor_optimizer_state,
        #    target_critic_params=target_critic_params,
        #    target_actor_params=target_actor_params,
        #    replay_buffer=replay_buffer,
        #    key=key,
        #)

        # Initial training state
        emitter_state = CustomQualityPGEmitterState(
            #critic_params=None, #critic_params,
            #critic_optimizer_state=None, #critic_optimizer_state,
            #actor_params=None, #actor_params,
            #actor_opt_state=None, #actor_optimizer_state,
            #target_critic_params=None, #target_critic_params,
            #target_actor_params=None, #target_actor_params,
            #replay_buffer=buffer,
            buffer_state=buffer_state,
            key=key,
        )

        return emitter_state

    def emit(  # type: ignore
        self,
        repertoire: GARepertoire,
        emitter_state: CustomQualityPGEmitterState,
        key: RNGKey,
    ) -> Tuple[Genotype, ExtraScores]:
        """Do a step of PG emission.

        Args:
            repertoire: the current repertoire of genotypes
            emitter_state: the state of the emitter used
            key: a random key

        Returns:
            A batch of offspring, a empty dict for signature.
        """

        batch_size = self._config.env_batch_size

        # sample parents
        mutation_pg_batch_size = int(batch_size) #int(batch_size - 1)
        parents = repertoire.select(
            key, mutation_pg_batch_size, selector=self._selector
        ).genotypes

        # apply the pg mutation
        offsprings_pg = self.emit_pg(emitter_state, parents)

        # get the actor (greedy actor)
        #offspring_actor = self.emit_actor(emitter_state)

        # add dimension for concatenation
        #offspring_actor = jax.tree.map(
        #    lambda x: jnp.expand_dims(x, axis=0), offspring_actor
        #)
        # gather offspring
        #genotypes = jax.tree.map(
        #    lambda x, y: jnp.concatenate([x, y], axis=0),
        #    offsprings_pg,
        #    offspring_actor,
        #)
        genotypes = offsprings_pg

        return genotypes, {}

    def emit_pg(
        self, emitter_state: CustomQualityPGEmitterState, parents: Genotype
    ) -> Genotype:
        """Emit the offsprings generated through pg mutation.

        Args:
            emitter_state: current emitter state, contains critic and
                replay buffer.
            parents: the parents selected to be applied gradients in order
                to mutate towards better performance.

        Returns:
            A new set of offsprings.
        """

        # create a batch of policy optimizer states
        policy_opt_states = jax.vmap(self._policies_optimizer.init)(parents)

        # prepare the batched policy update function with vmapping
        batched_policy_update_fn = jax.vmap(
            partial(self._update_policy, emitter_state=emitter_state), #critic_params=emitter_state.critic_params),
            in_axes=(0, 0)#, None),
        )

        def scan_update_policies(
            carry: Tuple[Params, optax.OptState, RNGKey],
            _: None,
        ) -> Tuple[Tuple[Params, optax.OptState, RNGKey], Any]:

            # Unpack the carry
            (policy_params, policy_opt_state, key) = carry
            key, subkey = jax.random.split(key)

            # sample a mini-batch of data from the replay-buffer
            #transitions = emitter_state.replay_buffer.sample(
            #    subkey, self._config.batch_size
            #)

            #transitions = emitter_state.replay_buffer.sample(emitter_state.buffer_state, key).experience
            (
                new_policy_params,
                new_policy_opt_states,
            ) = batched_policy_update_fn(policy_params, policy_opt_state)#transitions)
            return (new_policy_params, new_policy_opt_states, key), ()

        (
            final_policy_params,
            final_policy_opt_state,
            final_key,
        ), _ = jax.lax.scan(
            scan_update_policies,
            (parents, policy_opt_states, emitter_state.key),
            length=self._config.num_pg_training_steps,
        )

        return final_policy_params

    def emit_actor(self, emitter_state: CustomQualityPGEmitterState) -> Genotype:
        """Emit the greedy actor.

        Simply needs to be retrieved from the emitter state.

        Args:
            emitter_state: the current emitter state, it stores the
                greedy actor.

        Returns:
            The parameters of the actor.
        """
        return emitter_state.actor_params

    def state_update(  # type: ignore
        self,
        emitter_state: CustomQualityPGEmitterState,
        repertoire: Optional[Repertoire],
        genotypes: Optional[Genotype],
        fitnesses: Optional[Fitness],
        descriptors: Optional[Descriptor],
        extra_scores: ExtraScores,
    ) -> CustomQualityPGEmitterState:
        """This function gives an opportunity to update the emitter state
        after the genotypes have been scored.

        Here it is used to fill the Replay Buffer with the transitions
        from the scoring of the genotypes, and then the training of the
        critic/actor happens. Hence the params of critic/actor are updated,
        as well as their optimizer states.

        Args:
            emitter_state: current emitter state.
            repertoire: the current genotypes repertoire
            genotypes: unused here - but compulsory in the signature.
            fitnesses: unused here - but compulsory in the signature.
            descriptors: unused here - but compulsory in the signature.
            extra_scores: extra information coming from the scoring function,
                this contains the transitions added to the replay buffer.

        Returns:
            New emitter state where the replay buffer has been filled with
            the new experienced transitions.
        """
        # get the transitions out of the dictionary
        assert "transitions" in extra_scores.keys(), "Missing transitions or wrong key"
        transitions = extra_scores["transitions"]

        def prepare_for_buffer(x):
            # x: (P, T, E, *feat_dims)
            P, T, E, *feat_dims = x.shape

            # First move to (P, E, T, *feat_dims)
            x = x.transpose(0, 2, 1, *range(3, x.ndim))

            # Merge P * E into batch
            x = x.reshape(P * E, T, *feat_dims)

            # Add dummy seq_len=1 in the middle -> (batch, 1, T, feat_dim...)
            x = x[:, np.newaxis, ...]

            return x
        
        buffer_traj_batch = jax.tree.map(
            prepare_for_buffer,
            transitions
        )
        #buffer_traj_batch = jax.tree.map(
        #    lambda x: jnp.swapaxes(x, 0, 1)[
        #        :, np.newaxis
        #    ],  # put the batch dim first and add a dummy sequence dim
        #    transitions,
        #)  # (num_envs, 1, time_steps, ...)
        #buffer_traj_batch = jax.tree.map(
        #    lambda x: jnp.swapaxes(
        #        # Reorder from (num_policies, time_steps, num_envs, …)
        #        # → (time_steps, num_policies, num_envs, …)
        #        x.transpose(1, 0, 2, *range(3, x.ndim))
        #        # Merge num_policies * num_envs into one dimension
        #        .reshape(x.shape[1], -1, *x.shape[3:]),
        #        0, 1  # swap (time_steps, merged_envs, …) → (merged_envs, time_steps, …)
        #    )[:, np.newaxis],  # add dummy sequence dim
        #    transitions,
        #)
        new_buffer_state = self._buffer.add(emitter_state.buffer_state, buffer_traj_batch)
        final_emitter_state = emitter_state.replace(buffer_state=new_buffer_state)

        # add transitions in the replay buffer
        #emitter_state = emitter_state.replace(
        #    replay_buffer=emitter_state.replay_buffer.insert(transitions)
        #)
        
        # Conduct Actor-Critic training
        #final_emitter_state, _ = jax.lax.scan(
        #    self._scan_actor_critic_training,
        #    emitter_state,
        #    length=self._actor_critic_iterations,
        #)

        return final_emitter_state  # type: ignore

    def _update_policy(
        self,
        policy_params: Params,
        policy_opt_state: optax.OptState,
        #transitions: QDTransition,
        emitter_state: CustomQualityPGEmitterState,
        #critic_params: Params,
    ) -> Tuple[optax.OptState, Params]:
        """
        Perform one step of PG update on the off-spring policy.
        This function is vmapped to mutate the entire batch of off-springs
        in parallel.

        Args:
            policy_params: the parameters of the policy.
            policy_opt_state: the optimiser state of the policy.
            transitions: a mini-batch of transitions for gradient computation
            critic_params: the parameters of the critic networks serving as
                a differentiable target. This is fixed in each iteration.

        Returns:
            new_policy_params: new policy parameters
            new_policy_opt_state: updated optimiser state
        """
        new_target_params = policy_params

        # SAMPLE minibatches for each grad update
        key = emitter_state.key
        minibatches_list = []

        for _ in range(self._config.target_update_interval):
            key, subkey = jax.random.split(key)
            minibatch = self._buffer.sample(emitter_state.buffer_state, subkey).experience
            minibatches_list.append(minibatch)

        # Stack into [num_updates, ...] pytree
        minibatches = jax.tree.map(lambda *xs: jnp.stack(xs), *minibatches_list)

        # Update key back into emitter_state
        emitter_state = emitter_state.replace(key=key)

        def _apply_grad_update(carry, minibatch):
            # Compute the policy gradient
            policy_params, policy_opt_state = carry

            policy_gradient = jax.grad(self._policy_loss_fn)(
                policy_params,
                #critic_params,
                #transitions,
                new_target_params,
                #emitter_state
                minibatch
            )
            # Apply the update on the policy
            (
                policy_updates,
                new_policy_opt_state,
            ) = self._policies_optimizer.update(policy_gradient, policy_opt_state)
            new_policy_params = optax.apply_updates(policy_params, policy_updates)

            return (new_policy_params, new_policy_opt_state), ()
        
        (new_policy_params, new_policy_opt_state), _ = jax.lax.scan(
            _apply_grad_update, 
            (policy_params, policy_opt_state), 
            #length=config["TARGET_UPDATE_INTERVAL"]
            minibatches,
            )
        
        new_target_params = optax.incremental_update(
                        new_policy_params, #train_state.params,
                        new_target_params, #train_state.target_network_params,
                        self._config.tau,
                    )

        return new_target_params, new_policy_opt_state

In [14]:
class myPGAMEEmitter(MultiEmitter):
    def __init__(
        self,
        config: PGAMEConfig,
        policy_network: nn.Module,
        env: QDEnv,
        variation_fn: Callable[[Params, Params, RNGKey], Tuple[Params, RNGKey]],
        selector: Optional[Selector] = None,
    ) -> None:

        self._config = config
        self._policy_network = policy_network
        self._env = env
        self._variation_fn = variation_fn

        ga_batch_size = int(self._config.proportion_mutation_ga * config.env_batch_size)
        qpg_batch_size = config.env_batch_size - ga_batch_size

        @dataclass
        class CustomQualityPGConfig(QualityPGConfig):
            num_envs: Any = None
            num_steps: Any = None
            target_update_interval: Any = None
            tau: Any = None
                
        qpg_config = CustomQualityPGConfig(
            env_batch_size=qpg_batch_size,
            num_critic_training_steps=config.num_critic_training_steps,
            num_pg_training_steps=config.num_pg_training_steps,
            replay_buffer_size=config.replay_buffer_size,
            critic_hidden_layer_size=config.critic_hidden_layer_size,
            critic_learning_rate=config.critic_learning_rate,
            actor_learning_rate=config.greedy_learning_rate,
            policy_learning_rate=config.policy_learning_rate,
            noise_clip=config.noise_clip,
            policy_noise=config.policy_noise,
            discount=config.discount,
            reward_scaling=config.reward_scaling,
            batch_size=config.batch_size,
            soft_tau_update=config.soft_tau_update,
            policy_delay=config.policy_delay,

            num_envs = config.num_envs,
            num_steps = config.num_steps,
            target_update_interval = config.target_update_interval,
            tau = config.tau,
        )

        # define the quality emitter
        q_emitter = myQualityPGEmitter(
            config=qpg_config,
            policy_network=policy_network,
            env=env,
            selector=selector,
        )

        # define the GA emitter
        ga_emitter = MixingEmitter(
            mutation_fn=lambda x, r: (x, r),
            variation_fn=variation_fn,
            variation_percentage=1.0,
            batch_size=ga_batch_size,
            selector=selector,
        )

        super().__init__(emitters=(q_emitter, ga_emitter))

In [15]:
proportion_mutation_ga = 0.5 

# IQL/TD3 params
env_batch_size = batch_size #100 
replay_buffer_size = config["BUFFER_SIZE"] #1000000 
critic_hidden_layer_size = (256, 256) 
critic_learning_rate = 3e-4 
greedy_learning_rate = 3e-4 
policy_learning_rate = 1e-3 
noise_clip = 0.5
policy_noise = 0.2 
discount = config["GAMMA"] #0.99 
reward_scaling = 1.0 
transitions_batch_size = config["BUFFER_BATCH_SIZE"] #256 
soft_tau_update = 0.005
num_critic_training_steps = 300 
num_pg_training_steps = 100 
policy_delay = 2

# Define the PG-emitter config
@dataclass
class CustomPGAMEConfig(PGAMEConfig):
    num_envs: Any = None
    num_steps: Any = None
    target_update_interval: Any = None
    tau: Any = None

pga_emitter_config = CustomPGAMEConfig(
    env_batch_size=env_batch_size,
    batch_size=transitions_batch_size,
    proportion_mutation_ga=proportion_mutation_ga,
    critic_hidden_layer_size=critic_hidden_layer_size,
    critic_learning_rate=critic_learning_rate,
    greedy_learning_rate=greedy_learning_rate,
    policy_learning_rate=policy_learning_rate,
    noise_clip=noise_clip,
    policy_noise=policy_noise,
    discount=discount,
    reward_scaling=reward_scaling,
    replay_buffer_size=replay_buffer_size,
    soft_tau_update=soft_tau_update,
    num_critic_training_steps=num_critic_training_steps,
    num_pg_training_steps=num_pg_training_steps,
    policy_delay=policy_delay,

    num_envs = config["NUM_ENVS"],
    num_steps = config["NUM_STEPS"],
    target_update_interval = config["TARGET_UPDATE_INTERVAL"],
    tau = config["TAU"],
)

In [16]:
# Prepare the scoring function
# descriptor_extraction_fn = environments.descriptor_extractor[env_name] # Need to write my own function to extract descriptors
scoring_fn = functools.partial(
    scoring_function,
    episode_length=episode_length,
    play_reset_fn=reset_fn,
    play_step_fn=play_step_fn,
    descriptor_extractor=descriptor_extraction_fn,
)

# Get minimum reward value to make sure qd_score are positive
#reward_offset = environments.reward_offset[env_name]

# Define a metrics function
metrics_function = functools.partial(
    default_qd_metrics,
    qd_offset=1#reward_offset * episode_length, # Used to ensure QD score is positive could set to 1 if not needed
)

In [17]:
# Define emitter
variation_fn = functools.partial(
    isoline_variation, iso_sigma=iso_sigma, line_sigma=line_sigma
)

mixing_emitter = MixingEmitter(
    mutation_fn=None,
    variation_fn=variation_fn,
    variation_percentage=1.0,
    batch_size=batch_size
)

pg_emitter = myPGAMEEmitter(
    config=pga_emitter_config,
    policy_network=policy_network,
    env=wrapped_env, #env,
    variation_fn=variation_fn,
)

In [18]:
# Instantiate MAP-Elites
map_elites = MAPElites(
    scoring_function=scoring_fn,
    emitter=pg_emitter, #mixing_emitter,
    metrics_function=metrics_function,
)

# Compute the centroids
key, subkey = jax.random.split(key)
centroids = compute_cvt_centroids(
    num_descriptors=number_of_descriptors,#env.descriptor_length, # Num of dimensions in the descriptor
    num_init_cvt_samples=num_init_cvt_samples,
    num_centroids=num_centroids,
    minval=min_descriptor,
    maxval=max_descriptor,
    key=subkey,
)

print("Centroids:", centroids.shape)   # how many?
print("Descriptor dim:", centroids.shape[-1])

param_bytes = sum(x.size * x.dtype.itemsize for x in jax.tree.leaves(network_params))
print("Network params size (MB):", param_bytes / 1e6)

total_repertoire_mem_mb = param_bytes/1e6 * centroids.shape[0]
print("Expected repertoire memory (MB):", total_repertoire_mem_mb)

# Compute initial repertoire and emitter state
key, subkey = jax.random.split(key)
repertoire, emitter_state, init_metrics = map_elites.init(network_params, centroids, subkey)

Centroids: (2000, 2)
Descriptor dim: 2
Network params size (MB): 29.98528
Expected repertoire memory (MB): 59970.56
hstate shape: (5, 1, 128)
_obs shape: (5, 1, 1, 132)
_dones shape: (5, 1, 1)
new_hstate shape: (5, 1, 128)
actions {'ally_0': Traced<ShapedArray(int32[1])>with<DynamicJaxprTrace>, 'ally_1': Traced<ShapedArray(int32[1])>with<DynamicJaxprTrace>, 'ally_2': Traced<ShapedArray(int32[1])>with<DynamicJaxprTrace>, 'ally_3': Traced<ShapedArray(int32[1])>with<DynamicJaxprTrace>, 'ally_4': Traced<ShapedArray(int32[1])>with<DynamicJaxprTrace>}
data.reward shape: (64, 128, 5)
data.global_done shape: (64, 128, 5)
mask shape: (64, 128, 5)
fitness shape: (64, 1)
descriptors shape: (64, 2)
Init sample_traj shape: Timestep(obs={'__all__': (128, 120), 'ally_0': (128, 132), 'ally_1': (128, 132), 'ally_2': (128, 132), 'ally_3': (128, 132), 'ally_4': (128, 132)}, actions={'ally_0': (128,), 'ally_1': (128,), 'ally_2': (128,), 'ally_3': (128,), 'ally_4': (128,)}, rewards={'__all__': (128,), 'all

In [ ]:
log_period = 10
num_loops = num_iterations // log_period

# Initialize metrics
metrics = {key: jnp.array([]) for key in ["iteration", "qd_score", "coverage", "max_fitness", "time"]}

# Set up init metrics
init_metrics = jax.tree.map(lambda x: jnp.array([x]) if x.shape == () else x, init_metrics)
init_metrics["iteration"] = jnp.array([0], dtype=jnp.int32)
init_metrics["time"] = jnp.array([0.0])  # No time recorded for initialization

# Convert init_metrics to match the metrics dictionary structure
metrics = jax.tree.map(lambda metric, init_metric: jnp.concatenate([metric, init_metric], axis=0), metrics, init_metrics)

# Initialize CSV logger
#csv_logger = CSVLogger(
#    "mapelites-logs.csv",
#    header=list(metrics.keys())
#)

# Log initial metrics
#csv_logger.log(jax.tree.map(lambda x: x[-1], metrics))

# Main loop
map_elites_scan_update = map_elites.scan_update
print(jax.tree_util.tree_structure((repertoire, emitter_state, key)))

for i in range(num_loops):
    start_time = time.time()
    (
        repertoire,
        emitter_state,
        key,
    ), current_metrics = jax.lax.scan(
        map_elites_scan_update,
        (repertoire, emitter_state, key),
        (),
        length=log_period,
    )
    timelapse = time.time() - start_time
    print("num loop:", num_loops)

    # Metrics
    current_metrics["iteration"] = jnp.arange(1+log_period*i, 1+log_period*(i+1), dtype=jnp.int32)
    current_metrics["time"] = jnp.repeat(timelapse, log_period)
    metrics = jax.tree.map(lambda metric, current_metric: jnp.concatenate([metric, current_metric], axis=0), metrics, current_metrics)

    # Log
    #csv_logger.log(jax.tree.map(lambda x: x[-1], metrics))

PyTreeDef((CustomNode(MapElitesRepertoire[((),)], [{'params': {'Dense_0': {'bias': *, 'kernel': *}, 'Dense_1': {'bias': *, 'kernel': *}, 'ScannedRNN_0': {'GRUCell_0': {'hn': {'bias': *, 'kernel': *}, 'hr': {'kernel': *}, 'hz': {'kernel': *}, 'in': {'bias': *, 'kernel': *}, 'ir': {'bias': *, 'kernel': *}, 'iz': {'bias': *, 'kernel': *}}}}}, *, {}, *, *]), CustomNode(MultiEmitterState[()], [(CustomNode(CustomQualityPGEmitterState[('buffer_state', 'key')], [CustomNode(TrajectoryBufferState[('current_index', 'experience', 'is_full')], [*, CustomNode(Timestep[('actions', 'avail_actions', 'dones', 'obs', 'rewards')], [{'ally_0': *, 'ally_1': *, 'ally_2': *, 'ally_3': *, 'ally_4': *}, {'ally_0': *, 'ally_1': *, 'ally_2': *, 'ally_3': *, 'ally_4': *}, {'__all__': *, 'ally_0': *, 'ally_1': *, 'ally_2': *, 'ally_3': *, 'ally_4': *}, {'__all__': *, 'ally_0': *, 'ally_1': *, 'ally_2': *, 'ally_3': *, 'ally_4': *}, {'__all__': *, 'ally_0': *, 'ally_1': *, 'ally_2': *, 'ally_3': *, 'ally_4': *}]), *

2025-07-22 15:56:28.834895: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3021] Can't reduce memory use below 7.87GiB (8455146708 bytes) by rematerialization; only reduced to 10.19GiB (10942518209 bytes), down from 10.23GiB (10984469573 bytes) originally


num loop: 122


In [ ]:
# Create the x-axis array
#env_steps = metrics["iteration"]
env_steps = num_iterations * batch_size * episode_length * config["NUM_ENVS"]
#print(jnp.max(metrics["iteration"]))

%matplotlib inline
# Create the plots and the grid
fig, axes = plot_map_elites_results(env_steps=env_steps, metrics=metrics, repertoire=repertoire, min_descriptor=min_descriptor, max_descriptor=max_descriptor)

In [ ]:
best_idx = jnp.argmax(repertoire.fitnesses)
best_fitness = jnp.max(repertoire.fitnesses)
best_descriptor = repertoire.descriptors[best_idx]
print(
    f"Best fitness in the repertoire: {best_fitness:.2f}\n",
    f"Descriptor of the best individual in the repertoire: {best_descriptor}\n",
    f"Index in the repertoire of this individual: {best_idx}\n"
)

In [ ]:
# select the parameters of the best individual
my_params = jax.tree.map(
    lambda x: x[best_idx],
    repertoire.genotypes
)

In [ ]:
def preprocess_obs_with_id(obs_dict, env):
    """Simulate CTRolloutManager's preprocessing by adding one-hot agent IDs."""
    new_obs_dict = {}
    num_agents = len(env.agents)
    for i, agent in enumerate(env.agents):
        obs = obs_dict[agent].flatten()
        one_hot = jax.nn.one_hot(i, num_classes=num_agents)
        new_obs_dict[agent] = jnp.concatenate([obs, one_hot])
    return new_obs_dict


def visualize_recurrent_policy(trained_params, env, config):
    rng = jax.random.PRNGKey(config["SEED"])
    rng, reset_rng = jax.random.split(rng)
    #wrapped_env = CTRolloutManager(env, batch_size=1)

    # Create policy network
    #network = RNNQNetwork(
    #    action_dim=wrapped_env.max_action_space,
    #    hidden_dim=config["HIDDEN_SIZE"],
    #)
    network = RNNQNetwork(
        action_dim=env.action_space(env.agents[0]).n,
        hidden_dim=config["HIDDEN_SIZE"],
    )
    
    # Reset environment
    #obs, env_state = wrapped_env.batch_reset(reset_rng)
    obs, env_state = env.reset(reset_rng)
    #dones = {
    #    agent: jnp.zeros((1), dtype=bool)
    #    for agent in env.agents + ["__all__"]
    #}
    dones = {agent: jnp.array(False) for agent in env.agents}
    hstate = ScannedRNN.initialize_carry(
        config["HIDDEN_SIZE"], len(env.agents), 1
    )
    
    # Collect all environment states
    returns = {agent: 0.0 for agent in env.agents}
    state_seq = []
    max_steps = config["NUM_STEPS"]

    for step in range(max_steps):
        # Compute Q-values
        # Prepare inputs for network
        obs = preprocess_obs_with_id(obs, env)
        _obs = batchify(obs)         # (num_agents, obs_dim)
        _obs = _obs[:, None, :]                      # (num_agents, 1, obs_dim)

        #_dones = batchify(dones)    # (num_agents,)
        #_dones = _dones[:, None]                     # (num_agents, 1)
        _dones = jnp.stack([jnp.array([dones[agent]]) for agent in env.agents])  # shape (num_agents, 1)
        _dones = jnp.expand_dims(_dones, axis=-1)  # from (3, 1) to (3, 1, 1)

        #print("_obs.shape:", _obs.shape)
        #print("_dones.shape:", _dones.shape)
        #print("hstate.shape:", hstate.shape)

        def apply_fn(h, o, d):
            return network.apply(trained_params, h, o, d)

        hstate, q_vals = jax.vmap(apply_fn, in_axes=(0, 0, 0))(
            hstate,
            _obs,
            _dones,
        )
        #hstate = hstate[:, None, :]  # Already in (num_agents, hidden_dim)
        q_vals = q_vals.squeeze(axis=1)  # (num_agents, num_envs, num_actions) remove the time dim
        #print("q_vals.shape", q_vals.shape)
        
        actions = {}
        #avail_actions = wrapped_env.get_valid_actions(env_state.env_state)
        avail_actions = env.get_avail_actions(env_state.env_state)

        for i, agent in enumerate(env.agents):
            avail_agent = avail_actions[agent][None, None, :]  # shape (1, 1, n_actions)
            #print("avail_agent.shape", avail_agent.shape)
            
            unavail_actions = 1 - avail_agent  # shape (1, 1, n_actions)
            
            # Select Q-values for this agent only
            q_agent = q_vals[i][None, None, :]  # shape (1, 1, n_actions)
            q_masked = q_agent - (unavail_actions * 1e10)

            action = jnp.argmax(q_masked, axis=-1)  # shape (1, 1)
            action = action.squeeze()               # scalar
            #print("action.shape", action.shape)

            # Wrap in array with batch dim
            actions[agent] = int(action)    # shape (1,)
        
        rng, rng_s = jax.random.split(rng)
        state_seq.append((rng_s, env_state.env_state, actions))

        # Step environment

        # Batch the actions dict
        # Original actions: {'ally_0': 4, 'ally_1': 4, 'ally_2': 4}
        #actions = {k: jnp.array([v]) for k, v in actions.items()}

        #obs, env_state, rewards, dones, infos = wrapped_env.batch_step(
        #    rng_s, env_state, actions
        #)
        obs, env_state, rewards, dones, infos = env.step(rng_s, env_state, actions)
        returns = {a: returns[a] + rewards[a] for a in env.agents}
        
        if dones["__all__"]:
            break

    # Visualization
    print("Returns:", returns)

    viz = SMAXVisualizer(env, state_seq)
    viz.animate(view=False, save_fname="trained_iql_rnn.gif")

In [ ]:
scenario = map_name_to_scenario(config["MAP_NAME"])
env = HeuristicEnemySMAX(scenario=scenario, **config["ENV_KWARGS"])
env = SMAXLogWrapper(env)

visualize_recurrent_policy(my_params, env, config)